In [1]:
## read features:
import numpy as np
import pandas as pd

data_dir="/home/xin/Dropbox/BrainImaging_ML/data"
atlas_dir="/home/xin/Dropbox/BrainImaging_ML/Atlas_label"
work_dir="/home/xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/"

# figure_dir = "/home/xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/figures_sigmoid2_hc2"
# result_dir = "/home/xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/result_sigmoid2_hc2"

# with robust scale:
# figure_dir = "/home/xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/figures_ridge_robustscale_hc2"
# result_dir = "/home/xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/result_ridge_robustscale_hc2"

# with outlier removed and use spm8 vbm features:
# figure_dir = "/home/xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/figures_ridge_spmvbm_hc2"
# result_dir = "/home/xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/result_ridge_spmvbm_hc2"

# ransac/huber with spm8 vbm features:
figure_dir = "/home/xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/figures01_cca_spmvbm_hc2"
result_dir = "/home/xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/result01_cca_spmvbm_hc2"

import os
from os import path
#trying to make shift_graphs directory if it does not already exist:

if not os.path.exists(figure_dir):
    os.mkdir(figure_dir)
    
if not os.path.exists(result_dir):
    os.mkdir(result_dir)


# vbm analysis of cat12, 856 subjects. 
#It contains subjects in the ptsd group. So we need to remove them in training stage.
# cat_vbm = pd.read_csv(data_dir+"/ROI_catROI_neuromorphometrics_Vgm.csv", delimiter = ",")
# drop features with all zero values:
# cat_vbm.drop(columns = ['lInfLatVen', 'lOC'], inplace = True)
# print("cat_vbm:")
# print(cat_vbm.shape)
# # print(cat_vbm[:5])
# vbm = cat_vbm

spm_vbm_aal = pd.read_csv(data_dir+"/GMV_AAL_AvgExtract.csv", delimiter = ",")
# rename the subject id:
spm_vbm_aal.iloc[:,0].replace(to_replace=\
                              r'/home/xin/BrainImaging2016/Gender/Result03_Nifti_T1/VBM8/resampled_img/rm0wp1',
                              value='', regex=True, inplace = True)

spm_vbm_aal.iloc[:,0].replace(to_replace=\
                              r'.nii.gz',
                              value='', regex=True, inplace = True)

spm_vbm_aal.rename(columns = {'Unnamed: 0':'names'}, inplace = True) 
spm_vbm_aal['names']=spm_vbm_aal['names'].astype(int)
# spm_vbm_aal.iloc[:,1:] = spm_vbm_aal.iloc[:,1:].apply(pd.to_numeric, errors='coerce')

# read roi labels for aal atlas:
aal_label = pd.read_csv(atlas_dir+"/AAL_116_Label.csv", delimiter = ',')
spm_vbm_aal.columns = ['names'] + aal_label['ROIname'].to_list()
vbm = spm_vbm_aal


## read label FA features:
label_fa = pd.read_csv(data_dir+"/WMlabelResults_FA_all.csv", index_col=[0])
label_fa = label_fa.drop(['Unnamed: 0.1'], axis = 1)

# remove the last column which is empty.
num_col_fa=len(label_fa.columns) 
label_fa=label_fa.iloc[:,:num_col_fa-1]

print("label fa:")
print(label_fa.shape)
# print(label_fa[:5])


## read tract FA features:
tract_fa = pd.read_csv(data_dir+"/WMtractResults_FA_all.csv", index_col=0)
tract_fa = tract_fa.drop(['Unnamed: 0.1'], axis = 1)
# remove the last column which is empty.
num_col_fa=len(tract_fa.columns) 
tract_fa=tract_fa.iloc[:,:num_col_fa-1]

print("tract fa:")
print(tract_fa.shape)
# print(tract_fa[:5])


# multimodal_features = pd.concat([cat_vbm, label_fa, tract_fa, label_md, tract_md, alff, reho], axis = 1,)

# multimodal_features_keys = pd.concat([cat_vbm.drop(['names'], axis=1), label_fa, tract_fa, label_md, tract_md, alff, reho], axis = 1,
#                              keys = ['gmv', 'label_fa', 'tract_fa', 'label_md', 'tract_md', 'alff', 'reho'])

multimodal_features = pd.concat([vbm, label_fa, tract_fa], axis = 1,)

multimodal_features_keys = pd.concat([vbm.drop(['names'], axis=1), label_fa, tract_fa], axis = 1,
                             keys = ['gmv', 'label_fa', 'tract_fa'])


subject_id = vbm['names'].to_frame()

multimodal_features_keys = multimodal_features_keys.columns.get_level_values(0)


multimodal_features_keys
brain_columns = multimodal_features.columns

label fa:
(856, 50)
tract fa:
(856, 20)


In [2]:
brain_columns[1:]

Index(['Left Precentral', 'Right Precentral', 'Left Superior Frontal',
       'Right Superior Frontal', 'Left Superior Frontal Orbital',
       'Right Superior Frontal Orbital', 'Left Middle Frontal',
       'Right Middle Frontal', 'Left Middle Frontal Orbital',
       'Right Middle Frontal Orbital',
       ...
       ' Inferior.fronto-occipital.fasciculus.L',
       ' Inferior.fronto-occipital.fasciculus.R',
       ' Inferior.longitudinal.fasciculus.L',
       ' Inferior.longitudinal.fasciculus.R',
       ' Superior.longitudinal.fasciculus.L',
       ' Superior.longitudinal.fasciculus.R', ' Uncinate.fasciculus.L',
       ' Uncinate.fasciculus.R',
       ' Superior.longitudinal.fasciculus.(temporal.part).L',
       ' Superior.longitudinal.fasciculus.(temporal.part).R'],
      dtype='object', length=186)

In [3]:
cog_label_acc = [
    # executive control:
    "pcet_acc2",
    "pcpt_t_tp",
    "lnb_tp2",
    # episodic memory task:
    "pwmt_kiwrd_tot",
    "pfmt_ifac_tot",
    "volt_svt",
    # complex cognitive task:
    "pvrt_cr",
    "pmat_cr",
    "plot_tc",
    # social cognition task:
    "peit_cr",
    "pedt_a",
    "padt_a"
]

cog_label_speed = [
    # executive control:
    "pcet_rtcr",
    "pcpt_t_tprt",
    "lnb_rtc2",
    # episodic memory task:
    "pwmt_kiwrd_rtc",
    "pfmt_ifac_rtc",
    "volt_svtcrt",
    # complex cognitive task:
    "pvrt_rtcr",
    "pmat_rtcr",
    "plot_tcrt",
    # social cognition task:
    "peit_crt",
    "pedt_t",
    "padt_t",
    # sensorimotor speed task:
    "mp_mp2rtcr",
    "tap_tot"
]

cog_label_acc2 = [
    # executive control:
    "ABF",
    "ATT",
    "WM",
    # episodic memory task:
    "VMEM",
    "FMEM",
    "SMEM",
    # complex cognitive task:
    "LAN",
    "NVR",
    "SPA",
    # social cognition task:
    "EID",
    "EDI",
    "ADI"
]

cog_label_speed2 = [
    # executive control:
    "ABF",
    "ATT",
    "WM",
    # episodic memory task:
    "VMEM",
    "FMEM",
    "SMEM",
    # complex cognitive task:
    "LAN",
    "NVR",
    "SPA",
    # social cognition task:
    "EID",
    "EDI",
    "ADI",
    # sensorimotor speed task:
    "MOT",
    "SM"
]

cog_score_label = cog_label_acc + cog_label_speed
cog_score_label2 = [s + '_Accuracy' for s in cog_label_acc2] + [s + '_Speed' for s in cog_label_speed2]
num_cog = len(cog_score_label)

# select mental healthy subjects:
behav_data = pd.read_csv(data_dir+"/pyresult01_pnc_behav_mental3.csv", delimiter = ",")
hc_subject = behav_data.loc[behav_data['mental_hc3']==True, 'SUBJID']

behav_data = behav_data.loc[behav_data["SUBJID"].isin(hc_subject), ['SUBJID', 'Sex', 'age_at_cnb'] 
                            + cog_score_label]

#subject_info_feature['Sex'] = subject_info_feature['Sex'].apply({'F':0, 'M':1}.get)
# code females as -1 for easy interpretation of beta for age and age2
behav_data['Sex'] = behav_data['Sex'].apply({'F':-1, 'M':1}.get)

# reverse speed scores:
behav_data.loc[:,cog_label_speed] = behav_data.loc[:,cog_label_speed]*-1

#normalize the feature columns:
behav_data.loc[:,cog_score_label] = (behav_data.loc[:,cog_score_label] - behav_data.loc[:,cog_score_label].min()) / \
                        (behav_data.loc[:,cog_score_label].max() - behav_data.loc[:,cog_score_label].min())

num_subjects = behav_data.shape[0]

behav_data = behav_data.dropna(axis='rows')

# behav_data

In [4]:
# merge behav and brain features:
data_merge = behav_data.merge(multimodal_features, right_on = 'names', left_on = 'SUBJID')
# data_merge

In [5]:
data_merge_behav = data_merge[cog_score_label]
data_merge_brain = data_merge[brain_columns[1:]]
data_merge_behav

,pcet_acc2,pcpt_t_tp,lnb_tp2,pwmt_kiwrd_tot,pfmt_ifac_tot,volt_svt,pvrt_cr,pmat_cr,plot_tc,peit_cr,...,pfmt_ifac_rtc,volt_svtcrt,pvrt_rtcr,pmat_rtcr,plot_tcrt,peit_crt,pedt_t,padt_t,mp_mp2rtcr,tap_tot
0,0.789099,0.763636,0.6,0.961538,0.750000,0.500000,0.933333,0.791667,0.714286,0.72,...,0.598716,0.557628,0.823969,0.548683,0.809823,0.671786,0.703156,0.418923,0.857001,0.569551
1,0.265316,0.836364,0.8,0.884615,0.416667,0.928571,0.866667,0.375000,0.238095,0.76,...,0.868541,0.713976,0.820002,0.912164,0.907420,0.856878,0.800964,0.753083,0.864918,0.547645
2,0.885264,0.890909,0.8,1.000000,0.583333,0.857143,0.733333,0.375000,0.380952,0.80,...,0.938105,0.759731,0.801262,0.952719,0.858391,0.884682,0.790202,0.784144,0.910193,0.626506
3,0.303168,0.945455,0.7,0.807692,0.666667,0.857143,0.933333,0.416667,0.571429,0.80,...,0.776188,0.738155,0.879416,0.971467,0.897358,0.923871,0.857239,0.810571,0.882731,0.492881
4,0.701231,0.727273,0.5,0.653846,0.125000,0.428571,0.466667,0.291667,0.285714,0.56,...,0.808006,0.805703,0.732988,0.933971,0.812706,0.641599,0.717056,0.798042,0.633350,0.569551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,0.658006,0.927273,0.8,0.923077,0.791667,0.642857,0.733333,0.250000,0.428571,0.68,...,0.922876,0.670389,0.867458,0.917528,0.874897,0.841785,0.788969,0.779902,0.896338,0.598028
275,0.660220,0.436364,0.8,1.000000,0.875000,0.857143,0.800000,0.500000,0.333333,0.76,...,0.823126,0.361704,0.726622,0.790775,0.803624,0.774394,0.721877,0.710343,0.920584,0.442497
276,0.775045,0.927273,0.8,0.884615,0.583333,0.714286,0.733333,0.416667,0.190476,0.80,...,0.740672,0.643500,0.794952,0.897849,0.737080,0.725804,0.839079,0.725090,0.793419,0.497262
277,0.711536,0.763636,0.8,0.846154,0.958333,0.928571,0.733333,0.583333,0.714286,0.52,...,0.897204,0.725903,0.703026,0.884484,0.803826,0.832782,0.792781,0.768352,0.860960,0.615553


In [6]:
import rcca

from sklearn.model_selection import train_test_split

nIterations = 1000
all_numCC = np.zeros((nIterations,))
all_reg = np.zeros((nIterations,))

X = data_merge_brain.values
Y = data_merge_behav.values

# Analysis
# Initialize a cca object as an instantiation of the CCACrossValidate class.
ccaCV = rcca.CCACrossValidate(kernelcca=False, numCCs = [1, 2, 3, 4], regs = [0., 1e2, 1e4, 1e6], verbose = False)

ccaCV.train([X, Y])

# for ii in range(nIterations):
    
#     X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=ii)
    
    
#     # Use the train() and validate() methods to run the analysis and perform cross-dataset prediction.
#     ccaCV.train([X_train, Y_train])
    
#     all_numCC[ii] = ccaCV.best_numCC
#     all_reg[ii] = ccaCV.best_reg
        
        

/home/xin/environments/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


LinAlgError: the leading minor of order 107 of 'b' is not positive definite. The factorization of 'b' could not be completed and no eigenvalues or eigenvectors were computed.

In [7]:
from sklearn.cross_decomposition import CCA
from sklearn.model_selection import GridSearchCV

parameters = {'n_components': [2,3,4,5,6]}

cca = CCA()
clf = GridSearchCV(cca, parameters)
clf.fit(X, Y)


/home/xin/environments/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/xin/environments/lib/python3.6/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/home/xin/environments/lib/python3.6/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_averag

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=CCA(copy=True, max_iter=500, n_components=2, scale=True,
                           tol=1e-06),
             iid='warn', n_jobs=None,
             param_grid={'n_components': [2, 3, 4, 5, 6]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [8]:
clf.best_estimator_
X_c, Y_c = clf.best_estimator_.transform(X, Y)

In [11]:
X_c

array([[-1.04847660e-01, -1.79104915e-01],
       [ 2.43438259e-01,  2.45727400e-01],
       [-2.32205792e-01,  2.46142921e-02],
       [ 2.20470028e-01,  2.51873799e-02],
       [-7.69478365e-01,  2.89020563e-01],
       [ 8.55232763e-02, -1.98085658e-01],
       [-3.27434333e-01, -4.05216948e-02],
       [-1.44141365e-01,  1.11202754e-01],
       [ 1.06272798e-01, -1.08901055e-02],
       [-1.41477181e-01,  2.46396564e-01],
       [-6.00527016e-02,  7.95487173e-02],
       [ 1.10786016e-01,  1.09526808e-01],
       [-5.04439948e-01, -5.82330202e-02],
       [ 4.60951062e-02,  9.24184631e-02],
       [-2.50609215e-01,  1.19789053e-01],
       [ 4.02435114e-01,  1.91307301e-01],
       [ 2.45640328e-01,  5.50490741e-01],
       [ 5.65462088e-02,  2.46832585e-01],
       [ 4.50022325e-01,  8.23261415e-03],
       [-4.39456834e-02, -8.77208361e-02],
       [-5.17914764e-01,  1.88929909e-01],
       [-3.58734429e-01, -8.53955733e-02],
       [-8.16055445e-02, -8.24802849e-02],
       [-1.

In [13]:
clf.best_estimator_.x_loadings_

array([[-1.2600657 , -0.4302542 ],
       [-1.10308946, -0.34904353],
       [-1.35147292, -0.35817194],
       [-0.90302228, -0.14294541],
       [-1.52784426, -0.14290678],
       [-1.32711033, -0.23081717],
       [-1.60207906, -0.27160015],
       [-1.52492813, -0.24394698],
       [-1.32350421, -0.11381417],
       [-1.37295931,  0.11836032],
       [-1.36517855, -0.3416914 ],
       [-1.11672431, -0.18388821],
       [-1.55961621, -0.18534523],
       [-1.47083046, -0.2892913 ],
       [-1.37106265, -0.13123268],
       [-1.38021213,  0.07044506],
       [-1.19040956, -0.4748089 ],
       [-1.33174312, -0.67972788],
       [-1.04320526, -0.53557363],
       [-1.21509275, -0.39474371],
       [-0.86204007, -0.44062285],
       [-0.57355669, -0.69806645],
       [-1.43433811,  0.064708  ],
       [-1.40458665,  0.06466786],
       [-1.40459457, -0.25218964],
       [-1.24824423, -0.02032606],
       [-1.1823084 , -0.09017543],
       [-1.38158207, -0.36894171],
       [-1.01490968,

In [9]:
Yh = clf.best_estimator_.predict(X)

In [10]:
X.shape

(279, 212)